# Cline Prompt Learning Optimization on SWE-bench - Act Mode

<p align="left">
  <img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-logo.png" width="600" />
  <img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/cline.png" width="200" />
</p>

This notebook demonstrates how we used Prompt Learning to optimize Cline's performance on the SWE-bench dataset in **Act Mode**. Cline is a popular and powerful open-source coding agent. We look to improve its performance on SWE-bench by optimizing its **rules**, which are user specified instructions that Cline appends to its system prompt. 

[More on Cline](https://www.google.com/search?q=cline&sca_esv=764700c983d0c1df&sxsrf=AE3TifOTqxMNetNu45T7bn53deGE6bPn3w%3A1759280858717&ei=2n7caKLCK6XK0PEPpuCv6AY&ved=0ahUKEwiil5P154GQAxUlJTQIHSbwC20Q4dUDCBA&uact=5&oq=cline&gs_lp=Egxnd3Mtd2l6LXNlcnAiBWNsaW5lMgoQIxiABBgnGIoFMgoQIxiABBgnGIoFMgoQIxiABBgnGIoFMhMQLhiABBixAxjRAxhDGMcBGIoFMhMQLhiABBixAxjRAxgUGIcCGMcBMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgUQABiABDIFEAAYgAQyChAAGIAEGEMYigVIkw1Q8QNYpgxwA3gBkAEAmAG2AaABkAKqAQMxLjG4AQPIAQD4AQGYAgWgApwCwgIKEAAYsAMY1gQYR8ICDRAAGIAEGLADGEMYigXCAg4QABiwAxjkAhjWBNgBAcICDhAuGLADGLgGGMgD2AEBwgIQEC4YgAQY0QMYQxjHARiKBZgDAIgGAZAGE7oGBggBEAEYCZIHAzQuMaAHwhiyBwMxLjG4B5QCwgcDMC41yAcJ&sclient=gws-wiz-serp#:~:text=Cline%20%2D%20AI%20Coding,https%3A//cline.bot)

[More on Prompt Learning](https://arize.com/blog/prompt-learning-using-english-feedback-to-optimize-llm-systems/)

## Act Mode - Real Code Execution

Unlike Plan Mode, this notebook runs Cline in **Act Mode**, where Cline actually edits the codebase and generates patches. We then run the SWE-bench tests to compute a definitive accuracy of whether Cline made the correct edits. This provides ground truth evaluation of Cline's performance.

In Act Mode, Cline:
1. Analyzes the problem statement
2. Explores the codebase
3. Makes actual code edits
4. Generates patches
5. Has its patches validated against the SWE-bench test suite

## Setup

**Please visit README.md and complete all the Setup before running this notebook!**

## Important Note

Running this notebook is computationally intensive and expensive as it involves:
- Multiple API calls to Claude for each SWE-bench instance
- Actually cloning repositories and running tests in isolated environments
- Running SWE-bench harness to validate patches

Consider adjusting the training and test set sizes based on your requirements, budget constraints, and computational resources.


In [ ]:
from run_act import run_act
from swebench.harness.utils import load_swebench_dataset
import random
from evals_act import evaluate_results
import os
import sys
import subprocess
from pathlib import Path
import pandas as pd
import getpass

notebook_dir = Path().absolute()
sys.path.insert(0, str(notebook_dir.parent.parent))
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer

sys.path.insert(0, str(notebook_dir.parent))
from constants import CLINE_PROMPT

## API Keys

Set up your API keys for OpenAI, Anthropic, and Arize. If not already in your environment, you'll be prompted to enter them.


In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY") or getpass.getpass("Enter your Anthropic API key: ")
os.environ["ARIZE_API_KEY"] = os.getenv("ARIZE_API_KEY") or getpass.getpass("Enter your Arize API key: ")
SPACE_ID = os.getenv("SPACE_ID") or getpass.getpass("Enter your Arize space ID: ")

## Configuration

- LOOPS: number of Prompt Learning loops. How many times you want to optimize your prompt.
- TRAIN_SIZE: size of training set.
- TEST_SIZE: size of test set.
- WORKERS: SWE-bench is set up to run in parallel, with however many workers you specify. Set this relative to your machine's capabilities and your Claude rate limits.


In [ ]:
LOOPS = 5
TRAIN_SIZE = 150
TEST_SIZE = 150
WORKERS = 52

## Cline Environment Configuration

Set environment variables for Cline to run properly in Act Mode.


In [ ]:
os.environ["CLINE_DISABLE_TERMINAL_REUSE"] = "1"
os.environ["CLINE_DEFAULT_TERMINAL_PROFILE"] = "bash"
os.environ["CLINE_SHELL_INTEGRATION_TIMEOUT_SEC"] = "10"
os.environ["CLINE_STANDALONE_CAPTURE_STDIO"] = "1"
os.environ["CLINE_SKIP_RESUME_CONFIRMATION"] = "1"
os.environ["CLINE_AUTO_FOLLOWUP"] = "1"
os.environ["CLINE_ENVIRONMENT"] = "local"

## Train/Test Datasets

This code splits SWE-bench Lite into train/test splits.

The train set will be used to optimize the ruleset, while the test set will be used to measure the success of optimized rulesets.


In [ ]:
dataset_name = "SWE-bench/SWE-bench_Lite"
dataset = load_swebench_dataset(dataset_name, "test")
ids = [inst["instance_id"] for inst in dataset]
random.shuffle(ids)
train_ids = ids[:TRAIN_SIZE]        
test_ids = ids[len(ids) - TEST_SIZE:]
train_dataset = dataset[:TRAIN_SIZE]
test_dataset = dataset[len(dataset)-TEST_SIZE:]
train_pd = pd.DataFrame.from_dict(train_dataset)
test_pd = pd.DataFrame.from_dict(test_dataset)

## Upload Datasets to Arize

Upload datasets to Arize for experiment tracking and visualization.


In [ ]:
from arize.experimental.datasets import ArizeDatasetsClient
from arize.experimental.datasets.utils.constants import GENERATIVE

arize_client = ArizeDatasetsClient(api_key=os.getenv("ARIZE_API_KEY"))

# Prepare expanded dataset with more columns
def prepare_expanded_dataset(df):
    expanded_df = pd.DataFrame({
        'problem_statement': df['problem_statement'],
        'patch': df['patch'],
        'test_patch': df['test_patch'],
        'instance_id': df['instance_id'],
    })
    return expanded_df

# Prepare expanded datasets
train_expanded = prepare_expanded_dataset(train_pd)

# Upload expanded datasets to Arize
train_arize_id = arize_client.create_dataset(
    space_id=SPACE_ID,
    dataset_name="Cline Act Mode: SWE-bench Train",
    dataset_type=GENERATIVE,
    data=train_expanded
)

print("\nCreated train dataset with ID:", train_arize_id)

## Helper: Log Experiments to Arize

This helper function logs experiment results to Arize, allowing us to visualize and track optimization progress across iterations.


In [ ]:
from arize.experimental.datasets.experiments.types import (
    ExperimentTaskResultColumnNames,
    EvaluationResultColumnNames,
)

task_columns = ExperimentTaskResultColumnNames(
    example_id="example_id", result="final_plan"
)
evaluator_columns = EvaluationResultColumnNames(
    label="correctness",
    explanation="explanation",
    score="score"
)

# Get dataset with example_ids and merge with experiment results
def log_experiment_with_ids(client, space_id, experiment_name, experiment_df, dataset_name):
    # 1) fetch dataset and keep only instance_id -> id mapping
    dataset = client.get_dataset(space_id=space_id, dataset_name=dataset_name)
    id_map = dataset[['instance_id', 'id']].drop_duplicates()

    # 2) merge and build a minimal payload with only required columns
    merged = experiment_df.merge(id_map, on='instance_id', how='inner')
    payload = merged.rename(columns={"id": "example_id"})[
        ["example_id", "cline_patch", "correctness", "explanation", "score"]
    ].copy()

    return client.log_experiment(
        space_id=space_id,
        experiment_name=experiment_name,
        experiment_df=payload,
        dataset_name=dataset_name,
        task_columns=task_columns,
        evaluator_columns={'correctness': evaluator_columns},
    )

## Ruleset Optimization Loop

This is the main optimization loop. For each iteration:

1. **Run Cline in Act Mode on training set** with the current ruleset, generating actual code patches
2. **Run Cline in Act Mode on test set** with the current ruleset to measure generalization
3. **Run SWE-bench tests** to validate patches and compute pass/fail metrics
4. **Evaluate results** using LLM-as-judge to provide detailed feedback on patch quality
5. **Optimize the ruleset** using Prompt Learning based on training results and feedback
6. **Save results and rulesets** for tracking and analysis

The optimization loop uses actual test execution results (pass/fail) as ground truth, combined with LLM evaluator feedback to iteratively improve the ruleset.


In [ ]:
ruleset = ""

for loop in range(LOOPS):
    print(f"Running for loop: {loop}")

    train_run_id = f"claude_150_train_{loop}"
    test_run_id = f"150_act_test_{loop}"

    train_df = run_act(dataset_name=dataset_name, instance_ids=train_ids, run_id=train_run_id, ruleset=ruleset, workers=WORKERS)
    test_df = run_act(dataset_name=dataset_name, instance_ids=test_ids, run_id=test_run_id, ruleset=ruleset, workers=WORKERS)

    test_df.to_csv(f"act_results/test_results_{loop}.csv", index=False)
    
    train_acc = sum(train_df["pass_or_fail"] == "pass") / len(train_df)
    test_acc = sum(test_df["pass_or_fail"] == "pass") / len(test_df)
    print(f"Train Accuracy: {train_acc}")
    print(f"Test Accuracy: {test_acc}")

    # make sure any swebench package installations did not affect phoenix package
    subprocess.run([
        "/opt/anaconda3/envs/cline/bin/python3",
        "-m",
        "pip",
        "install",
        "--upgrade",
        "arize-phoenix",
        "wrapt",
    ])
    evaluated_train_results = evaluate_results(train_df)
    evaluated_train_results.to_csv(f"act_results/train_results_{loop}.csv", index=False)
    evaluated_train_results["score"] = [1.0 if x == "correct" else 0.0 for x in evaluated_train_results["correctness"]]
    log_experiment_with_ids(arize_client, SPACE_ID, f"Train {loop}", evaluated_train_results, "Cline Act Mode: SWE-bench Train")

    pl_optimizer = PromptLearningOptimizer(
        prompt=CLINE_PROMPT,
        model_choice="gpt-5",
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
    ruleset = pl_optimizer.optimize(
        dataset=evaluated_train_results,
        output_column="cline_patch",
        feedback_columns=["correctness", "explanation"],
        ruleset=ruleset,
        context_size_k=400000
    )
    with open(f"act_rulesets/ruleset_{loop}.txt", "w") as f:
        f.write(f"train_accuracy: {train_acc} \n")
        f.write(f"test_accuracy: {test_acc} \n")
        f.write(f"optimized ruleset_{loop}: \n {ruleset} \n")

